## 데이터 로드

In [1]:
import pandas as pd
import numpy as np
import json

with open('./data/train_original.json', encoding='utf-8') as train_f:
    train_data = json.loads(train_f.read())
    train_df = pd.DataFrame(train_data['documents'])

with open('./data/valid_original.json', encoding='utf-8') as valid_f:
    valid_data = json.loads(valid_f.read())
    valid_df = pd.DataFrame(valid_data['documents'])

In [6]:
train_df.text[0]

[[{'index': 0,
   'sentence': 'ha당 조사료 400만원…작물별 차등 지원',
   'highlight_indices': ''}],
 [{'index': 1, 'sentence': '이성훈 sinawi@hanmail.net', 'highlight_indices': ''}],
 [{'index': 2,
   'sentence': '전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 시행하는 쌀 생산조정제를 적극 추진키로 했다.',
   'highlight_indices': ''}],
 [{'index': 3,
   'sentence': '쌀 생산조정제는 벼를 심었던 논에 벼 대신 사료작물이나 콩 등 다른 작물을 심으면 벼와의 일정 소득차를 보전해주는 제도다.',
   'highlight_indices': '35,37'}],
 [{'index': 4,
   'sentence': '올해 전남의 논 다른 작물 재배 계획면적은 전국 5만ha의 약 21%인 1만 698ha로, 세부시행지침을 확정, 시군에 통보했다.',
   'highlight_indices': '9,11;33,34'},
  {'index': 5,
   'sentence': '지원사업 대상은 2017년산 쌀 변동직불금을 받은 농지에 10a(300평) 이상 벼 이외 다른 작물을 재배한 농업인이다.',
   'highlight_indices': '50,52'}],
 [{'index': 6,
   'sentence': '지원 대상 작물은 1년생을 포함한 다년생의 모든 작물이 해당되나 재배 면적 확대 시 수급과잉이 우려되는 고추, 무, 배추, 인삼, 대파 등 수급 불안 품목은 제외된다.',
   'highlight_indices': '24,26'}],
 [{'index': 7,
   'sentence': '농지의 경우도 이미 다른 작물 재배 의무가 부여된 간척지, 정부매입비축농지, 농진청 시범사업, 경관보전 직불금 수령 농지 등은 제외될 예정이다.',
   'highlight_ind

In [2]:
def sentence_validation(item, sentence):
    val_num = 0
    if item == []:
        val_num = 1
    if '@' in sentence or '/사진' in sentence:
        val_num = 1
    return val_num


def extract_body(article) -> str:
    item_list = []

    # item = train_df['text'][n][idx]
    for idx, item in enumerate(article):
        sentence = item[0]['sentence']
        if sentence_validation(item, sentence):
            # print(f'넘기는 항목 : {sentence}')
            continue
        
        if len(item) >= 2:
            for sub_item in item:
                # print(f'추가되는 하위 항목 : {sentence}')
                item_list.append(sub_item['sentence'])
                continue
        
        # print(f'추가되는 항목 : {sentence}')
        item_list.append(sentence)

    return item_list